**Описание проекта**

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('/datasets/insurance.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Данные загружены и изучены, пропусков и аномалий не обнаружено. Перейдем к дальнейшей работе.

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

Попробую переформулировать вопрос

Если в задаче линеной регрессии $a = Xw$ умножить $X (M, N)$ на обратимую матрицу $Z$, размерности $(N, N)$, то при вычислении весов $w = (X^T X)^{-1} X^T y$ предсказания $a$ не дожны изменится.

**Ответ:** Предсказания $a$ не изменятся

**Обоснование:** Для доказательства подставим вместо $X$ $XZ$

$$
a_{new} = XZw_{new}
$$$$
w_{new} = ((XZ)^T XZ)^{-1}(XZ)^T y
$$
подставим $w_{new}$ в $a_{new}$ и преобразуем:

$$
a_{new} = X Z ((XZ)^T XZ)^{-1}(XZ)^T y = \\X Z (Z^TX^T XZ)^{-1}Z^T X^T y = \\X Z (X^TXZ)^{-1} (Z^T)^{-1} Z^T X^T y = \\X Z Z^{-1}(X^TX)^{-1} (Z^T)^{-1} Z^T X^T y = \\X E (X^TX)^{-1} E  X^T y = \\X (X^TX)^{-1} X^T y =  X w
$$
Пояснения:

Раскрываем транспонирования по свойству: $(AB)^T = B^T A^T$
Считаем $X^TX$ - квадратная обратимая марица, $Z$ - обратимая по опеределнию, то раскрываем скобки по свойству $(AB)^{-1} = B^{-1} A^{-1}$ 2 раза

Используя ассоциативность умножения матриц и определение обратной матрицы получим ответ.

Таким образом получаем что:

$$
a_{new} = a
$$


## 3. Алгоритм преобразования

**Алгоритм**

1. Выделяем целевые признаки и целевой признак.
2. Разделяем выборку на тренировочную и тестовую.
3. Определяем численные признаки для скалирования.
4. Преобразуем признаки тренировочной и тестовой выборок в массивы NumPy.
5. Генерируем случайную квадратную матрицу преобразований размером равным количеству признаков.
6. Проводим проверку обратимости полученной матрицы вычислением ее определителя.
7. Если определитель не равен нулю:
 - Умножаем матрицы признаков тренировочной и тестовой выборок на матрицу преобразований.
 - Проводим скалирование матрицы признаков тренировочной и тестовой выборок.
 - Обучаем модель на тренировочной выборке и получаем предсказания на тестовой.  
8. Если определитель равен нулю - генерируем новую матрицу преобразований и переходим на п.6.

**Обоснование**

Качество линейной регрессии не поменяется согласно п.2

## 4. Проверка алгоритма

In [5]:
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']

In [6]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, random_state=12345)

In [7]:
numeric = features.drop('Пол', axis=1).columns
numeric

Index(['Возраст', 'Зарплата', 'Члены семьи'], dtype='object')

In [8]:
def linreg(X_train , X_test, y_train, y_test):
    scaler = StandardScaler()
    scaler.fit(X_train.loc[:,numeric])
    X_train.loc[:,numeric] = scaler.transform(X_train.loc[:,numeric])
    X_test.loc[:,numeric] = scaler.transform(X_test.loc[:,numeric])
    
    #Обучаем модель линейной регрессии
    model = LinearRegression()
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    return r2_score(y_test, predict)

In [9]:
R2_before = linreg(features_train, features_test, target_train, target_test)
R2_before

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


0.4305278542485148

In [10]:
X_train, X_test = features_train.values, features_test.values
print(X_train)
print(X_test)

[[ 0.         -0.23862317 -1.98861705 -1.08239912]
 [ 0.          0.70548434 -0.26239699  0.75707499]
 [ 0.          0.35144402 -0.7671397   0.75707499]
 ...
 [ 1.          1.17753809  0.48462222 -0.16266207]
 [ 0.         -1.06471724  1.02974435  2.5965491 ]
 [ 0.         -1.41875756  0.09092291 -1.08239912]]
[[ 0.00000000e+00  2.33430582e-01 -9.07844658e-02  2.59654910e+00]
 [ 0.00000000e+00  2.23965904e+00  3.23104556e-01  7.57074988e-01]
 [ 1.00000000e+00  9.41511213e-01  2.22156014e-01 -1.08239912e+00]
 ...
 [ 1.00000000e+00 -5.92663489e-01  2.62535431e-01 -1.08239912e+00]
 [ 0.00000000e+00 -2.59629565e-03  2.82725139e-01  2.59654910e+00]
 [ 1.00000000e+00  9.41511213e-01 -1.41258737e-01 -1.62662068e-01]]


In [11]:
def random_matrix(X_train, X_test, count = 100):
    """
    Функция генерирует случайную квадратную матрицу размера равного количеству признаков
    Умножает её на матрицу признаков, обучает модель линейной регрессии на преобразованных данных и 
    считает для них метрику R2.
    На выходе получаем значение метрики и матрицу преобразования
    """
    n = X_train.shape[1]
    A = np.random.RandomState().randint(100, size=(n, n))
    flag = 0
    while flag == 0:
        if np.linalg.det(A) != 0:
            A_train = X_train @ A
            A_test = X_test @ A
            A_train = pd.DataFrame(A_train, index = features_train.index, columns = features_train.columns)
            A_test = pd.DataFrame(A_test, index = features_test.index, columns = features_test.columns)
            r2 = linreg(A_train, A_test, target_train, target_test)
            flag = 1
            print('Матрица преобразований:')
            print(A)
            print('Значение метрики R2 равно:', r2)
    return r2, A        

In [12]:
R2_after = random_matrix(X_train, X_test)[0]

Матрица преобразований:
[[16 85 67 80]
 [48 86 67 62]
 [65 59 14 64]
 [41 35 17 33]]
Значение метрики R2 равно: 0.4305278542485147


In [13]:
R2_before - R2_after

1.1102230246251565e-16

По результату работы видим, что качество линейной регрессии до и после не сильно отличается.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования